In [90]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 24: Blizzard Basin ---</h2><p>With everything replanted for next year (and with elephants and monkeys to tend the grove), you and the Elves leave for the extraction point.</p>
<p>Partway up the mountain that shields the grove is a flat, open area that serves as the extraction point. It's a bit of a climb, but nothing the expedition can't handle.</p>
<p>At least, that would normally be true; now that the mountain is covered in snow, things have become more difficult than the Elves are used to.</p>
<p>As the expedition reaches a valley that must be traversed to reach the extraction site, you find that strong, turbulent winds are pushing small <em>blizzards</em> of snow and sharp ice around the valley. It's a good thing everyone packed warm clothes! To make it across safely, you'll need to find a way to avoid them.</p>
<p>Fortunately, it's easy to see all of this from the entrance to the valley, so you make a map of the valley and the blizzards (your puzzle input). For example:</p>
<pre><code>#.#####
#.....#
#&gt;....#
#.....#
#...v.#
#.....#
#####.#
</code></pre>
<p>The walls of the valley are drawn as <code>#</code>; everything else is ground. Clear ground - where there is currently no blizzard - is drawn as <code>.</code>. Otherwise, blizzards are drawn with an arrow indicating their direction of motion: up (<code>^</code>), down (<code>v</code>), left (<code>&lt;</code>), or right (<code>&gt;</code>).</p>
<p>The above map includes two blizzards, one moving right (<code>&gt;</code>) and one moving down (<code>v</code>). In one minute, each blizzard moves one position in the direction it is pointing:</p>
<pre><code>
#.#####
#.....#
#.&gt;...#
#.....#
#.....#
#...v.#
#####.#
</code></pre>

<p>Due to <span title="I think, anyway. Do I look like a theoretical blizzacist?">conservation of blizzard energy</span>, as a blizzard reaches the wall of the valley, a new blizzard forms on the opposite side of the valley moving in the same direction. After another minute, the bottom downward-moving blizzard has been replaced with a new downward-moving blizzard at the top of the valley instead:</p>
<pre><code>#.#####
#...v.#
#..&gt;..#
#.....#
#.....#
#.....#
#####.#
</code></pre>
<p>Because blizzards are made of tiny snowflakes, they pass right through each other. After another minute, both blizzards temporarily occupy the same position, marked <code>2</code>:</p>
<pre><code>#.#####
#.....#
#...2.#
#.....#
#.....#
#.....#
#####.#
</code></pre>
<p>After another minute, the situation resolves itself, giving each blizzard back its personal space:</p>
<pre><code>#.#####
#.....#
#....&gt;#
#...v.#
#.....#
#.....#
#####.#
</code></pre>
<p>Finally, after yet another minute, the rightward-facing blizzard on the right is replaced with a new one on the left facing the same direction:</p>
<pre><code>#.#####
#.....#
#&gt;....#
#.....#
#...v.#
#.....#
#####.#
</code></pre>
<p>This process repeats at least as long as you are observing it, but probably forever.</p>
<p>Here is a more complex example:</p>
<pre><code>#.######
#&gt;&gt;.&lt;^&lt;#
#.&lt;..&lt;&lt;#
#&gt;v.&gt;&lt;&gt;#
#&lt;^v^^&gt;#
######.#
</code></pre>
<p>Your expedition begins in the only non-wall position in the top row and needs to reach the only non-wall position in the bottom row. On each minute, you can <em>move</em> up, down, left, or right, or you can <em>wait</em> in place. You and the blizzards act <em>simultaneously</em>, and you cannot share a position with a blizzard.</p>
<p>In the above example, the fastest way to reach your goal requires <code><em>18</em></code> steps. Drawing the position of the expedition as <code>E</code>, one way to achieve this is:</p>
<pre><code>Initial state:
#<em>E</em>######
#&gt;&gt;.&lt;^&lt;#
#.&lt;..&lt;&lt;#
#&gt;v.&gt;&lt;&gt;#
#&lt;^v^^&gt;#
######.#

Minute 1, move down:
#.######<br/>#<em>E</em>&gt;3.&lt;.#
#&lt;..&lt;&lt;.#
#&gt;2.22.#
#&gt;v..^&lt;#
######.#

Minute 2, move down:
#.######
#.2&gt;2..#<br/>#<em>E</em>^22^&lt;#
#.&gt;2.^&gt;#
#.&gt;..&lt;.#
######.#

Minute 3, wait:
#.######
#&lt;^&lt;22.#<br/>#<em>E</em>2&lt;.2.#
#&gt;&lt;2&gt;..#
#..&gt;&lt;..#
######.#

Minute 4, move up:
#.######<br/>#<em>E</em>&lt;..22#
#&lt;&lt;.&lt;..#
#&lt;2.&gt;&gt;.#
#.^22^.#
######.#

Minute 5, move right:
#.######
#2<em>E</em>v.&lt;&gt;#
#&lt;.&lt;..&lt;#
#.^&gt;^22#
#.2..2.#
######.#

Minute 6, move right:
#.######
#&gt;2<em>E</em>&lt;.&lt;#
#.2v^2&lt;#
#&gt;..&gt;2&gt;#
#&lt;....&gt;#
######.#

Minute 7, move down:
#.######
#.22^2.#
#&lt;v<em>E</em>&lt;2.#
#&gt;&gt;v&lt;&gt;.#
#&gt;....&lt;#
######.#

Minute 8, move left:
#.######
#.&lt;&gt;2^.#
#.<em>E</em>&lt;&lt;.&lt;#
#.22..&gt;#
#.2v^2.#
######.#

Minute 9, move up:
#.######
#&lt;<em>E</em>2&gt;&gt;.#
#.&lt;&lt;.&lt;.#
#&gt;2&gt;2^.#
#.v&gt;&lt;^.#
######.#

Minute 10, move right:
#.######
#.2<em>E</em>.&gt;2#
#&lt;2v2^.#
#&lt;&gt;.&gt;2.#
#..&lt;&gt;..#
######.#

Minute 11, wait:
#.######
#2^<em>E</em>^2&gt;#
#&lt;v&lt;.^&lt;#
#..2.&gt;2#
#.&lt;..&gt;.#
######.#

Minute 12, move down:
#.######
#&gt;&gt;.&lt;^&lt;#
#.&lt;<em>E</em>.&lt;&lt;#
#&gt;v.&gt;&lt;&gt;#
#&lt;^v^^&gt;#
######.#

Minute 13, move down:
#.######
#.&gt;3.&lt;.#
#&lt;..&lt;&lt;.#
#&gt;2<em>E</em>22.#
#&gt;v..^&lt;#
######.#

Minute 14, move right:
#.######
#.2&gt;2..#
#.^22^&lt;#
#.&gt;2<em>E</em>^&gt;#
#.&gt;..&lt;.#
######.#

Minute 15, move right:
#.######
#&lt;^&lt;22.#
#.2&lt;.2.#
#&gt;&lt;2&gt;<em>E</em>.#
#..&gt;&lt;..#
######.#

Minute 16, move right:
#.######
#.&lt;..22#
#&lt;&lt;.&lt;..#
#&lt;2.&gt;&gt;<em>E</em>#
#.^22^.#
######.#

Minute 17, move down:
#.######
#2.v.&lt;&gt;#
#&lt;.&lt;..&lt;#
#.^&gt;^22#
#.2..2<em>E</em>#
######.#

Minute 18, move down:
#.######
#&gt;2.&lt;.&lt;#
#.2v^2&lt;#
#&gt;..&gt;2&gt;#
#&lt;....&gt;#<br/>######<em>E</em>#
</code></pre>

<p><em>What is the fewest number of minutes required to avoid the blizzards and reach the goal?</em></p>
</article>


In [91]:
from collections import deque
from collections.abc import Iterator
from copy import deepcopy
from pprint import pprint
from typing import Literal


tests = [
    {
        "name": "Smaller Example",
        "s": """
            #.#####
            #.....#
            #>....#
            #.....#
            #...v.#
            #.....#
            #####.#
       """,
        "expected": 10,
    },
    {
        "name": "Example",
        "s": """
            #.######
            #>>.<^<#
            #.<..<<#
            #>v.><>#
            #<^v^^>#
            ######.#
        """,
        "expected": 18,
    },
]

type State = list[list[list[str]]]
type Place = tuple[int, int]


class Valley(Str):
    def __init__(self, s: str) -> None:
        self.state = [
            [[] if c == "." else [c] for c in l.strip()] for l in s.strip().splitlines()
        ]

        self.rows, self.cols = len(self.state), len(self.state[0])

        self.start = 0, self.state[0].index([])
        self.goal = self.rows - 1, self.state[-1].index([])

    def next_state(self, state: State, old: State) -> tuple[State, State]:
        for r, c in product(range(1, self.rows - 1), range(1, self.cols - 1)):
            old[r][c].clear()

        for r, c in product(range(1, self.rows - 1), range(1, self.cols - 1)):
            for blizzard in state[r][c]:
                rr, cc = r, c
                if blizzard == "^":
                    rr -= 1
                elif blizzard == "v":
                    rr += 1
                elif blizzard == ">":
                    cc += 1
                elif blizzard == "<":
                    cc -= 1

                rr = 1 + (rr - 1) % (self.rows - 2)
                cc = 1 + (cc - 1) % (self.cols - 2)
                old[rr][cc].append(blizzard)

        return state, old

    def next_places(self, place: Place, state: State) -> Iterator[Place]:
        row, col = place
        for dr, dc in ((-1, 0), (0, 1), (1, 0), (0, -1), (0, 0)):
            r, c = row + dr, col + dc
            if 0 <= r < self.rows and 0 <= c < self.cols and not state[r][c]:
                yield r, c

    def min_minutes_to_goal(self, debug=False) -> int:
        minutes, _ = self.from_to(debug=debug)
        return minutes

    def from_to(
        self, me: Place = None, goal: Place = None, state: State = None, debug=False
    ) -> tuple[int, State]:
        state = deepcopy(self.state) if state is None else deepcopy(state)
        old = deepcopy(state)

        me = self.start if me is None else me
        goal = self.goal if goal is None else goal

        queue = deque([me])
        seen = set()
        minutes = 0

        while queue:
            old, state = self.next_state(state, old)
            h = self.hash_state(state)

            for _ in range(len(queue)):
                me = queue.popleft()

                if me == goal:
                    return minutes, old

                if (me, h) in seen:
                    continue

                seen.add((me, h))

                if debug:
                    print(f"Minute {minutes}:" if minutes else "Initial state:")
                    print(self.state_str(old, me))
                    print()

                queue.extend(self.next_places(me, state))

            minutes += 1

        return -1, None

    @staticmethod
    def hash_state(state: State) -> int:
        return hash(str(state))

    @classmethod
    def state_str(cls, state: State, me: Place = None) -> str:
        rows, cols = len(state), len(state[0])
        me = (0, 1) if me is None else me

        str_buffer = [["."] * cols for _ in range(rows)]

        for r, c in product(range(rows), range(cols)):
            if (r, c) == me:
                str_buffer[r][c] = "E"
            elif len(state[r][c]) == 1:
                str_buffer[r][c] = state[r][c][0]
            elif len(state[r][c]) > 1:
                str_buffer[r][c] = str(len(state[r][c]))

        return "\n".join("".join(row) for row in str_buffer)


@test(tests=tests)
def test_part_I(s: str) -> int:
    return Valley(s).min_minutes_to_goal()


Test Smaller Example passed, for test_part_I.
Test Example passed, for test_part_I.
Success


In [92]:
with open("../input/day24.txt") as f:
    puzzle = f.read()

print(f"Part II: {Valley(puzzle).min_minutes_to_goal()}")

Part II: 290


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>290</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>As the expedition reaches the far side of the valley, one of the Elves looks especially dismayed:</p>
<p>He <em>forgot his snacks</em> at the entrance to the valley!</p>
<p>Since you're so good at dodging blizzards, the Elves humbly request that you go back for his snacks. From the same initial conditions, how quickly can you make it from the start to the goal, then back to the start, then back to the goal?</p>
<p>In the above example, the first trip to the goal takes <code>18</code> minutes, the trip back to the start takes <code>23</code> minutes, and the trip back to the goal again takes <code>13</code> minutes, for a total time of <code><em>54</em></code> minutes.</p>
<p><em>What is the fewest number of minutes required to reach the goal, go back to the start, then reach the goal again?</em></p>
</article>


In [95]:
tests = [
    {
        "name": "Smaller Example",
        "s": """
            #.#####
            #.....#
            #>....#
            #.....#
            #...v.#
            #.....#
            #####.#
       """,
        "expected": 30,
    },
    {
        "name": "Example",
        "s": """
            #.######
            #>>.<^<#
            #.<..<<#
            #>v.><>#
            #<^v^^>#
            ######.#
        """,
        "expected": 54,
    },
]


class ValleyII(Valley):
    def go_back_go(self, debug=False) -> int:
        go, state = self.from_to(debug=debug)
        back, state = self.from_to(self.goal, self.start, state, debug)
        go_again, _ = self.from_to(self.start, self.goal, state, debug)
        return go + back + go_again


@test(tests=tests[:])
def test_part_II(s: str) -> int:
    return ValleyII(s).go_back_go()


Test Smaller Example passed, for test_part_II.
Test Example passed, for test_part_II.
Success


In [96]:
print(f"PartI II: {ValleyII(puzzle).go_back_go()}")

PartI II: 842


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>895</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
